In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
import numpy as np

In [2]:
parentDirectory = os.path.abspath(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir))
DATA_DIR = parentDirectory +'/data/'
FIGURES_DIR = parentDirectory +'/figures/'

df = pd.read_parquet(DATA_DIR+'dk_new_food_timeseries_items1.parquet')

In [3]:
selected_codes = ['FR','DE','US','IT','CA','GB','ES','AU','MX','BR','IN','DK']
categories = list(df['category'].unique())

In [4]:
len(categories)

28

In [5]:
len(selected_codes)

12

In [6]:
df = df.loc[df['name']!='Bánh mì']

In [7]:
weeks_2019 = list(df.iloc[:5]['ts'].apply(lambda x: pd.Series(x['max_ratio'])).iloc[0].index[:52])
weeks_2020 = list(df.iloc[:5]['ts'].apply(lambda x: pd.Series(x['max_ratio'])).iloc[0].index[52:])

In [8]:
items = df['mid'].unique()

In [9]:
list_items = []

for c,gr1 in df.groupby('category'):
    print(c)
    for item,gr2 in gr1.groupby('name'):
        gr2.loc[:,'total_2019'] = gr2['ts'].apply(lambda x: np.sum([x['max_ratio'][i] for i in weeks_2019])).copy()
        gr2.loc[:,'total_2020'] = gr2['ts'].apply(lambda x: np.sum([x['max_ratio'][i] for i in weeks_2020])).copy()
        gr2.loc[:,'total_2019_2020'] = gr2['ts'].apply(lambda x: np.sum([x['max_ratio'][i] for i in weeks_2019+weeks_2020])).copy()
        
        
        for cnt,row in gr2[['country_code','total_2019','total_2020','total_2019_2020']].iterrows():
            entry = {}
            entry['country'] = row['country_code']
            entry['category'] = c
            entry['total_2019'] = row['total_2019']
            entry['total_2020'] = row['total_2020']
            entry['total_2019_2020'] = row['total_2019_2020']
            entry['item'] = item
            list_items.append(entry)


beef dish


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


bread and flatbread
cheese
chicken dish
cocktail
dessert
egg dish
fish dish
fruit
herb
lamb dish
pasta, pizza and noodle dish
pastry and bakery product
pie
pork dish
potato dish
rice dish
salad
sandwich
sauce
sausage
snack
soft drink
soup
spice
stew
vegetable and legume
wine, beer and liquor


In [10]:
df_analysis = pd.DataFrame(list_items)

In [11]:
for c,gr in df_analysis.groupby('category'):
    print(c)
    
    
    for country, gr2 in gr.groupby('country'):
        y1 = gr2['total_2019'].rank(ascending = True).values
        y2 = gr2['total_2020'].rank(ascending = True).values
        y3 = gr2['total_2019_2020'].rank(ascending = True).values
        if country == 'AU':
            S1 = (pd.Series( y1, index = gr2['item'].values))
            S2 = (pd.Series( y2, index = gr2['item'].values))
            S3 = (pd.Series( y3, index = gr2['item'].values))
        else:
            S1 = S1.add(pd.Series( y1, index = gr2['item'].values), fill_value = 0)
            S2 = S2.add(pd.Series( y2, index = gr2['item'].values), fill_value = 0)
            S3 = S3.add(pd.Series( y3, index = gr2['item'].values), fill_value = 0)

            
    print('average top 10 2019-2020:')
    
    my_list = [i.split('(')[0].strip() for i in (list((S3/12).dropna().sort_values().index[-10:]))[::-1]]
    my_string = ', '.join(map(str, my_list)) 
   
    print(my_string)
    
    print('\n')
    print('all sorted by volume 2019-2020:')
    
    my_list = [i.split('(')[0].strip() for i in (list((S3/12).dropna().sort_values().index))[::-1]]
    my_string = ', '.join(map(str, my_list)) 
   
    print(my_string)
    
    
    print('---------------------------------------------')
    
    
    '''
    print('average top 10 2019:')
    
    my_list = [i.split('(')[0].strip() for i in (list((S1/12).dropna().sort_values().index[-10:]))[::-1]]
    my_string = ', '.join(map(str, my_list)) 
   
    print(my_string)
    print('---------------------------------------------')
    
    print('average top 10 2020:')
    
    
    my_list = [i.split('(')[0].strip() for i in (list((S2/12).dropna().sort_values().index[-10:]))[::-1]]
    my_string = ', '.join(map(str, my_list)) 
    
    print(my_string)
    print('---------------------------------------------')
    print('---------------------------------------------')
    print('\n')
    '''
    
    

beef dish
average top 10 2019-2020:
Hamburger, Beef, Steak, Meatball, Meatloaf, Beef Stroganoff, Beef mince, Fajita, Sirloin steak, Big Mac


all sorted by volume 2019-2020:
Hamburger, Beef, Steak, Meatball, Meatloaf, Beef Stroganoff, Beef mince, Fajita, Sirloin steak, Big Mac, Wagyu, Kofta, Cheeseburger, Brisket, Moussaka, Beef tenderloin, Prosciutto, Kobe beef, Corned beef, Spam, Carpaccio, Rib eye steak, T-bone steak, Whopper, Pastrami, Beef bourguignon, Beef Wellington, Beef tongue, Korean barbecue, Oxtail, Chuck steak, Yakiniku, Shabu-shabu, Strip steak, Round steak, Salisbury steak, Picadillo, Rouladen, Tournedos Rossini, Steak and kidney pie, Motsunabe, Bistek, Horumonyaki, Beef ball, Rollatini, Boeuf à la mode, Stovies, Vanillerostbraten, Bò 7 món
---------------------------------------------
bread and flatbread
average top 10 2019-2020:
Bread, Pita, Bagel, Sourdough, Baguette, Naan, Pretzel, Focaccia, Bruschetta, White bread


all sorted by volume 2019-2020:
Bread, Pita, Bagel

average top 10 2019-2020:
Cod, Caviar, Tuna, Salmon, Squid, Catfish, Tempura, Sardine, Smoked salmon, Crayfish


all sorted by volume 2019-2020:
Cod, Caviar, Tuna, Salmon, Squid, Catfish, Tempura, Sardine, Smoked salmon, Crayfish, Eel, Fish and chips, Anchovy, Ceviche, Sashimi, Mahi-mahi, Pufferfish, Fish finger, Cuttlefish, Sturgeon, Stingray, Marlin, Haddock, Fish sauce, Fishcake, Tataki, Surimi, California roll, Fugu, Red snapper, Lox, Fish ball, Taramasalata, Fried prawn, Crab cake, Kedgeree, Ayu, Brandade, Sakana, Acqua pazza, Chikuwa, Court-bouillon, Kabayaki, Canh chua, Bánh canh, Cacciucco, Pempek, Stargazy pie, Gianchetti, Run down, Vorschmack, Ikan bakar, Fish head curry, Amok trey, Pissalat, Schmaltz herring, Psarosoupa
---------------------------------------------
fruit
average top 10 2019-2020:
Apple, Lemon, Pineapple, Avocado, Grape, Cherry, Watermelon, Mango, Banana, Fruit


all sorted by volume 2019-2020:
Apple, Lemon, Pineapple, Avocado, Grape, Cherry, Watermelon, Mang

average top 10 2019-2020:
Soup, Broth, Ramen, Miso, Pho, French onion soup, Hot pot, Goulash, Minestrone, Cream of mushroom soup


all sorted by volume 2019-2020:
Soup, Broth, Ramen, Miso, Pho, French onion soup, Hot pot, Goulash, Minestrone, Cream of mushroom soup, Gazpacho, Miso soup, Consommé, Bisque, Pea soup, Noodle soup, Tom yum, Laksa, Hot and sour soup, Nabemono, Pozole, Bird's nest soup, Egg drop soup, Beef noodle soup, Menudo, Haleem, Panada, Soto, Bún bò Huế, Ribollita, Nihari, Chankonabe, Sopa de mondongo, Zōni, Tinola, Taco soup, Bún riêu, Cozido, Butajiru, Tong sui, Scotch broth, Garbure, Olla podrida, Mohinga, Ching bo leung, Oille, Chakna, Sour cherry soup, Alicot, Soup Number Five, Pig's organ soup, Samlar machu
---------------------------------------------
spice
average top 10 2019-2020:
Garlic, Table salt, Ginger, Chili pepper, Spice, Turmeric, Vanilla, Cinnamon, Common Fig, Black pepper


all sorted by volume 2019-2020:
Garlic, Table salt, Ginger, Chili pepper, Spic

In [12]:
check = 0

for c,gr in df_analysis.groupby('category'):
    print(c)
    
    
    for country, gr2 in gr.groupby('country'):
        y1 = gr2['total_2019'].rank(ascending = True).values
        y2 = gr2['total_2020'].rank(ascending = True).values
        y3 = gr2['total_2019_2020'].rank(ascending = True).values
        if country == 'AU':
            S1 = (pd.Series( y1, index = gr2['item'].values))
            S2 = (pd.Series( y2, index = gr2['item'].values))
            S3 = (pd.Series( y3, index = gr2['item'].values))
        else:
            S1 = S1.add(pd.Series( y1, index = gr2['item'].values), fill_value = 0)
            S2 = S2.add(pd.Series( y2, index = gr2['item'].values), fill_value = 0)
            S3 = S3.add(pd.Series( y3, index = gr2['item'].values), fill_value = 0)

            
    #print('average top 10 2019-2020:')
    
    my_list = [i.split('(')[0].strip() for i in (list((S3/12).dropna().sort_values().index[-10:]))[::-1]]
    my_string = ', '.join(map(str, my_list)) 
   
    
    
    my_list = [i.split('(')[0].strip() for i in (list((S3/12).dropna().sort_values().index))[::-1]]
    my_string = ', '.join(map(str, my_list)) 
   
    for i in list(S3.sort_values().index[::-1]):
        print(i)
    #print('---------------------------------------------')
    
    
    '''
    print('average top 10 2019:')
    
    my_list = [i.split('(')[0].strip() for i in (list((S1/12).dropna().sort_values().index[-10:]))[::-1]]
    my_string = ', '.join(map(str, my_list)) 
   
    print(my_string)
    print('---------------------------------------------')
    
    print('average top 10 2020:')
    
    
    my_list = [i.split('(')[0].strip() for i in (list((S2/12).dropna().sort_values().index[-10:]))[::-1]]
    my_string = ', '.join(map(str, my_list)) 
    
    print(my_string)
    print('---------------------------------------------')
    print('---------------------------------------------')
    print('\n')
    '''
    

beef dish
Hamburger
Beef
Steak
Meatball
Meatloaf
Beef Stroganoff
Beef mince
Fajita
Sirloin steak
Big Mac
Wagyu
Kofta
Cheeseburger
Brisket
Moussaka
Beef tenderloin
Prosciutto
Kobe beef
Corned beef
Spam
Carpaccio
Rib eye steak
T-bone steak
Whopper
Pastrami
Beef bourguignon
Beef Wellington
Beef tongue
Korean barbecue
Oxtail
Chuck steak
Yakiniku
Shabu-shabu
Strip steak
Round steak
Salisbury steak
Picadillo
Rouladen
Tournedos Rossini
Steak and kidney pie
Motsunabe
Bistek
Horumonyaki
Beef ball
Rollatini
Boeuf à la mode
Stovies
Vanillerostbraten
Bò 7 món
bread and flatbread
Bread (Literature Subject)
Pita
Bagel
Sourdough
Baguette
Naan
Pretzel
Focaccia
Bruschetta
White bread
Rye bread
Brown bread
Flatbread
Ciabatta
Breadstick
Roti
Challah
Papadum
Crumpet
English muffin
Matzo
Piadina
Bánh
Injera
Simit
Kulcha
Butterbrot
Murtabak
Pane carasau
Bazlama
cheese
Cheese (Ingredient)
Mozzarella
Cream cheese
Parmigiano-Reggiano
Ricotta
Feta
Fondue
Cheddar cheese
Mascarpone
Cottage cheese
Goat cheese
Brie

Pizza
Pasta
Spaghetti
Lasagne
Carbonara
Noodle (Ingredient)
Gnocchi
Macaroni
Penne
Ravioli
Wonton
Empanada
Pad Thai
Jiaozi
Tortellini
Linguine
Pizza delivery
Soba
Rice noodles
Dim sum food
Pierogi
Vermicelli
Cannelloni
Baozi
Calzone
Fettuccine
Rigatoni
Orzo
Fusilli
Instant noodles
Yakisoba
Cellophane noodles
Pappardelle
Farfalle
Spaghetti alle vongole
Spätzle
Dango (Japanese Dish)
Spaghetti aglio e olio
Capellini
Manti (Ingredient)
Bucatini
Xiaolongbao
Pastitsio
Lo mein
Pasta e fagioli
Conchiglie
Fregula
Pancit
Orecchiette
Mie goreng
Mandu
Agnolotti
Pici
Kraft Dinner
Cavatelli
Mentaiko
Chiburekki
Timballo
Tortelloni
Nikuman
Strozzapreti
Naporitan
Mafaldine
Varenyky
Bigoli
Gemelli
Maultasche
Fagottini
Ptitim
Pizzoccheri
Campanelle
Radiatori
Cepelinai
Bavette
SpaghettiOs
Passatelli
Lasagnette
Rotelle
Songpyeon
Anelli
Mì Quảng
Corzetti
Kreplach
Cincinnati chili
Acini di pepe
Trenette
Fiori
Farfel
Tripoline
Egg barley
Stringozzi
Occhi di Lupo
Pillus
pastry and bakery product
Baking powder
